In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's 
# several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import cupy as np
import cudf as pd
path='../input/house-prices-advanced-regression-techniques'
df_train=pd.read_csv(path+'/train.csv')
df_test=pd.read_csv(path+'/test.csv')

In [3]:
print(df_train.shape,df_test.shape)

(1460, 81) (1459, 80)


In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,12,2008,WD,Normal,250000


In [5]:
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(0.0)
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].fillna(0.0)

In [6]:
x = df_train[['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 
'YearBuilt', 'FullBath', 'OpenPorchSF', 'WoodDeckSF', '2ndFlrSF', 'YearRemodAdd', 
              'MasVnrArea', 'LotArea']]
#'LotFrontage'-dropped
y=df_train['SalePrice']
y=np.array(y)
y=y.reshape(-1,1)
y=y.astype('float64')

In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x.as_matrix())

In [8]:
print(x.dtype,y.dtype)

float64 float64


In [9]:
from sklearn.model_selection import train_test_split
# from cuml.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

In [10]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1314, 13)
(146, 13)
(1314, 1)
(146, 1)


In [11]:
import cuml
from cuml import LinearRegression
from cuml.metrics import accuracy_score
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
for i in algorithm:
    print("Algorithm:",i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(x_train,y_train)
    preds = lr.predict(x_test)
#     acc=accuracy_score(y_test, preds)
#     print(acc)
    mse=cuml.metrics.regression.mean_squared_error(y_test,preds)
    r2=cuml.metrics.regression.r2_score(y_test,preds)
    mae=cuml.metrics.regression.mean_absolute_error(y_test,preds)
    print("MSE:",mse)
    print("R2 Score:",r2)
    print("MAE:",mae)

Algorithm: svd
MSE: 18273279606.78335
R2 Score: 1.0
MAE: 91834.78710827547
Algorithm: eig
MSE: 18273279606.783325
R2 Score: 1.0
MAE: 91834.7871082754
Algorithm: qr
MSE: 18273279606.783356
R2 Score: 1.0
MAE: 91834.78710827547
Algorithm: svd-qr
MSE: 9141000639.117205
R2 Score: 0.0015542173682294713
MAE: 66007.99886308658
Algorithm: svd-jacobi
MSE: 9141000639.117205
R2 Score: 0.0015542173682294713
MAE: 66007.99886308658
